In [1]:
import ipyparallel as ipp
rc = ipp.Client()
view = rc[:]

In [2]:
%%px --local
from glob import glob
import SimpleITK as sitk
import numpy as np
from scipy.ndimage import zoom

In [3]:
%%px --local
KAGGLE_DATASET_PATH = "/mnt/Data/KAGGLE/"
LUNA_DATASET_PATH = "/mnt/Data/LUNA/"
EXPORT_IMAGE_FORMAT = ".nii.gz"
EXPORT_SPACING = np.array([1.0, 1.0, 1.0])  # (z, y, x). They HAVE to be floats

In [4]:
%%px --local
def get_id_luna_image(path):
    return path.split("/")[-1].split(".nii.gz")[0]

def get_id_equalized_luna_image(path):
    return path.split("/")[-1].split(EXPORT_IMAGE_FORMAT)[0]

def load_luna_image(path):
    return sitk.ReadImage(path)

def get_id_kaggle_image(path):
    return path.split("/")[-1]

def load_kaggle_image(path):    
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(path)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    return image

def get_id_kaggle_mask(path):
    return tuple(path.split("/")[-1].split(EXPORT_MASK_IMAGE_FORMAT)[0].split("_"))


In [5]:
# Lists all the images in the dataset
luna_image_paths = glob(LUNA_DATASET_PATH + "seg-nodules-LUNA16/*.nii.gz")
# This path should exist beforehand for the ImageWrite to work
export_luna_dataset_path = LUNA_DATASET_PATH + "equalized_spacing_nodule_masks/"
# List equalized images
luna_equalized_image_paths = glob(export_luna_dataset_path + "*" + EXPORT_IMAGE_FORMAT)

In [6]:
luna_ids = {get_id_luna_image(path) for path in luna_image_paths}
equalized_luna_ids = {get_id_equalized_luna_image(path) for path in luna_equalized_image_paths}
pending_ids = luna_ids - equalized_luna_ids
remaining_luna_image_paths = [path for path in luna_image_paths if get_id_luna_image(path) in pending_ids]

In [7]:
len(remaining_luna_image_paths)

0

In [8]:
def equalize_img_spacings(img_path, export_dataset_path, load_fn, get_id_fn):
    # Load CT scan
    img = load_fn(img_path)
    img_arr = sitk.GetArrayViewFromImage(img)
    img_spacing = img.GetSpacing()[::-1]  # since sitk is (x,y,z) but np.array (z,y,x)

    # Resize CT scan
    resize_factor = img_spacing / EXPORT_SPACING
    img_resized_arr = zoom(img_arr, resize_factor)

    # Set resized scan back as a SimpleITK image object
    img_resized = sitk.GetImageFromArray(img_resized_arr)
    img_resized.SetSpacing(EXPORT_SPACING)
    img_resized.SetOrigin(img.GetOrigin())

    # Write image to disk
    export_img_path = export_dataset_path + get_id_fn(img_path) + EXPORT_IMAGE_FORMAT
    sitk.WriteImage(img_resized, export_img_path)

In [9]:
image_paths = remaining_luna_image_paths
export_luna_dataset_paths = [export_luna_dataset_path] * len(image_paths)
load_luna_image_fns = [load_luna_image] * len(image_paths)
get_id_luna_image_fns = [get_id_luna_image] * len(image_paths)

view.map(
    equalize_img_spacings,
    image_paths,
    export_luna_dataset_paths,
    load_luna_image_fns,
    get_id_luna_image_fns
).wait()

AttributeError: 'list' object has no attribute 'wait'